In [1]:
# import pickle
# import numpy as np
# import pandas as pd
# from sklearn.neighbors import NearestNeighbors

In [2]:
# Cluster_folder_path = "./cluster_data/"

# def find_nearest_neighbor(encoded_try_list: np.ndarray, clustered_user_df: pd.DataFrame):
#     # Ensure the clustered_user_df has a 'user_id' column and encoded category columns.
#     if 'user_id' not in clustered_user_df.columns:
#         raise ValueError("clustered_user_df must include a 'user_id' column.")

#     # Select the encoded features from clustered_user_df; assume all columns except 'user_id'
#     features = clustered_user_df.drop(columns=['user_id', 'cluster']).values

#     # Create the NearestNeighbors model using Jaccard metric
#     nn_model = NearestNeighbors(n_neighbors=1, metric='jaccard')
#     nn_model.fit(features)

#     # Query the model to get nearest neighbor index and distance
#     distances, indices = nn_model.kneighbors(encoded_try_list)

#     # Create a list to hold results
#     results = []
#     for idx, (dist, ind) in enumerate(zip(distances, indices)):
#         # Get the corresponding user_id from clustered_user_df using the index
#         nearest_cluster_id = clustered_user_df.iloc[ind[0]]['cluster']
#         results.append({
#             'input_index': int(idx),
#             'nearest_cluster_id': int(nearest_cluster_id),
#             # 'distance': dist[0]
#         })
#     return results

# def find_cluster_id(categories: list):
#     clustered_user_df = pd.read_pickle(Cluster_folder_path + 'clustered_user_df.pkl')

#     if not isinstance(categories, list) or not all(isinstance(i, list) for i in categories):
#         categories = [categories]
#     categories = np.array(categories)

#     with open(Cluster_folder_path + 'users_categories_encoder.pkl', 'rb') as f:
#         user_category_encoder = pickle.load(f)
#         f.close()

#     encoded_try_list = user_category_encoder.transform(categories)

#     results = find_nearest_neighbor(encoded_try_list,clustered_user_df)
#     return results

In [3]:
from cluster_utils import *
from scipy.sparse import csr_matrix
from sparse_dot_topn import sp_matmul_topn
from sklearn.model_selection import train_test_split

In [4]:
# Define the database folder path and file names
db_folder = '../../data/processed_data/yelp_data/'
data_files = ['review']

# Load data into a dictionary
yelp_data = load_data_from_db(db_folder, data_files)

# Check loaded data
for table, df in yelp_data.items():
    print(f"Loaded {len(df)} rows from {table} table.")
    
df_review = yelp_data["review"][["user_id", "business_id", "stars"]]

Loaded 980418 rows from review table.


In [5]:
# Load cluster data and map to reviews (your existing code)
df_cluster = pd.read_excel("../data_processing/clustered_users.xlsx")
user_cluster = df_cluster.set_index('user_id')['cluster'].to_dict()
df_review['cluster'] = df_review['user_id'].map(user_cluster)
df_review = df_review.dropna(subset=['cluster'])

# Calculate the global mean of stars
global_mean = df_review['stars'].mean()

# Choose a constant C (e.g., 5; adjust based on your data)
C = 5

# Group by business_id and cluster, calculate sum and count of stars
df_cluster_review = df_review.groupby(['business_id', 'cluster'])['stars'].agg(['sum', 'count']).reset_index()

# Calculate the weighted score
df_cluster_review['score'] = (df_cluster_review['sum'] + C * global_mean) / (df_cluster_review['count'] + C)

cluster_business = df_cluster_review.copy()

C:\Users\yueny\AppData\Local\Temp\ipykernel_8940\1249627618.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['cluster'] = df_review['user_id'].map(user_cluster)


In [6]:
# sort by score
sorted_cluster_business = cluster_business.sort_values(by='score', ascending=False)
sorted_cluster_business[:100000]

,business_id,cluster,sum,count,score
574325,mtvT7uRey3F395STFRM1Tg,1090.0,99.0,20,4.731418
150966,C6YaSrMAzy3jJqinlFVudw,1830.0,89.0,18,4.708063
39331,2KIDQyTh-HzLxOUEDqtDBg,1090.0,131.0,27,4.696420
504036,gfLsBY-xsNE9-ktiTvTvGA,1356.0,84.0,17,4.694793
438558,atZ_olNKXOG4rEr6mccN8g,1090.0,112.0,23,4.688766
...,...,...,...,...,...
569134,mRTRMo-M0B_9fMOnzzMtsQ,978.0,5.0,1,4.047574
569136,mRTRMo-M0B_9fMOnzzMtsQ,1589.0,5.0,1,4.047574
569114,mRGoD7FolIbmR5ANRU3SCw,167.0,5.0,1,4.047574
569103,mRALAZduUGa_QBAyxQ89HQ,1084.0,5.0,1,4.047574


In [7]:
def get_cluster_business(cluster_business):
    cluster_mapping = {clus: idx for idx, clus in enumerate(cluster_business['cluster'].unique())}
    business_mapping = {biz: idx for idx, biz in enumerate(cluster_business['business_id'].unique())}    
    return cluster_mapping, business_mapping

In [8]:
cluster_mapping, business_mapping = get_cluster_business(cluster_business)

In [9]:
train_data, test_data = train_test_split(cluster_business, test_size=0.2, random_state=42)

cluster_business = train_data.copy()

In [10]:
# Map user_id and business_id to numerical indices
cluster_business['cluster_idx'] = cluster_business['cluster'].map(cluster_mapping)
cluster_business['business_idx'] = cluster_business['business_id'].map(business_mapping)

# Creating the sparse user-item interaction matrix using weighted_stars
user_item_sparse = csr_matrix(
    (cluster_business['score'], (cluster_business['cluster_idx'], cluster_business['business_idx'])),
    shape=(len(cluster_mapping), len(business_mapping))
)

# Replace NaN values in the sparse matrix
user_item_sparse.data = np.nan_to_num(user_item_sparse.data)

In [11]:
def sparse_cosine_similarity_topn(A, top_n, threshold=0):
    C = sp_matmul_topn(A.T, A.T, top_n=top_n, threshold=threshold, n_threads=4, sort=True)
    return C


In [12]:
# Compute item similarity
item_similarity_sparse = sparse_cosine_similarity_topn(user_item_sparse, top_n=100, threshold=0.01)

In [ ]:
def optimize_db(conn):
    """Apply SQLite performance optimizations."""
    cursor = conn.cursor()
    cursor.executescript('''
        PRAGMA synchronous = OFF;
        PRAGMA journal_mode = MEMORY;
        PRAGMA temp_store = MEMORY;
        PRAGMA cache_size = 1000000;
    ''')
    conn.commit()


def insert_cluster_item(cluster_business, conn, batch_size=50000):
    """Optimized batch insert for cluster-item interactions."""
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    total_records = len(cluster_business)
    # sort the score and filter the top 50 for each cluster
    cluster_business = cluster_business.sort_values(by='score', ascending=False).groupby('cluster').head(50)
    data = cluster_business[['cluster', 'business_id', 'score']].values.tolist()

    for i in range(0, total_records, batch_size):
        batch = data[i:i + batch_size]
        cursor.executemany('''INSERT OR IGNORE INTO cluster_item_index (cluster, business_id, score)
                              VALUES (?, ?, ?)''', batch)

        if i % (batch_size * 5) == 0:  # Commit every 5 batches
            conn.commit()
            print(f"Inserted {i + len(batch)} / {total_records} cluster-item records.")

    conn.commit()  # Final commit
    print(f"Total {total_records} cluster-item records inserted.")

    
def insert_item_vectors(item_similarity_sparse, business_mapping, conn, batch_size=5000, progress_interval=50000):
    """Optimized batch insert for item similarity vectors."""
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    total_inserted = 0
    batch = []
    business_keys = list(business_mapping.keys())  # Convert keys to list for faster indexing

    for row_idx in range(item_similarity_sparse.shape[0]):
        row_vector = item_similarity_sparse.getrow(row_idx)
        row_indices = row_vector.indices
        row_data = row_vector.data

        serialized_row = pickle.dumps((row_indices, row_data))
        item_id = business_keys[row_idx]  # Faster lookup

        batch.append((item_id, serialized_row))

        if len(batch) >= batch_size:
            cursor.executemany('''INSERT OR REPLACE INTO item_item_similarity (item_id, similarity_vector)
                                  VALUES (?, ?)''', batch)
            total_inserted += len(batch)

            if total_inserted % progress_interval == 0:
                print(f"Inserted {total_inserted} item vectors...")

            batch = []

    if batch:  # Insert remaining records
        cursor.executemany('''INSERT OR REPLACE INTO item_item_similarity (item_id, similarity_vector)
                              VALUES (?, ?)''', batch)
        total_inserted += len(batch)

    conn.commit()
    print(f"Total {total_inserted} item vectors inserted.")


def insert_mappings(business_mapping, conn, batch_size=50000):
    """Optimized batch insert for business mappings."""
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    data = list(business_mapping.items())
    total_records = len(data)

    for i in range(0, total_records, batch_size):
        batch = data[i:i + batch_size]
        cursor.executemany('''INSERT OR REPLACE INTO business_mapping (business_id, business_idx)
                              VALUES (?, ?)''', batch)

        if i % (batch_size * 5) == 0:  # Commit every 5 batches
            conn.commit()
            print(f"Inserted {i + len(batch)} / {total_records} business mappings.")

    conn.commit()
    print(f"Total {total_records} business mappings inserted.")
    
    
# function to insert df_cluster to the table user_cluster_mapping
def insert_user_cluster_mapping(df_cluster, conn, batch_size=50000):
    """Optimized batch insert for user cluster mapping."""
    cursor = conn.cursor()
    cursor.execute('BEGIN TRANSACTION')

    total_records = len(df_cluster)
    data = df_cluster[['user_id', 'cluster']].values.tolist()

    for i in range(0, total_records, batch_size):
        batch = data[i:i + batch_size]
        cursor.executemany('''INSERT OR IGNORE INTO user_cluster_mapping (user_id, cluster)
                              VALUES (?, ?)''', batch)

        if i % (batch_size * 5) == 0:  # Commit every 5 batches
            conn.commit()
            print(f"Inserted {i + len(batch)} / {total_records} user cluster mappings.")

    conn.commit()  # Final commit
    print(f"Total {total_records} user cluster mappings inserted.")


In [14]:
# Connect to SQLite (this will create a file-based database)
db_path = './cluster_data/yelp_ClusterItemCF.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
optimize_db(conn)

cursor.execute('''CREATE TABLE IF NOT EXISTS user_cluster_mapping (
    user_id TEXT PRIMARY KEY,
    cluster INTEGER
)''')

# Create tables for cluster-item and item-item indexes
cursor.execute('''CREATE TABLE IF NOT EXISTS cluster_item_index (
    cluster INTEGER,
    business_id TEXT,
    score REAL,
    PRIMARY KEY (cluster, business_id)
)''')

cursor.execute('''CREATE INDEX IF NOT EXISTS idx_cluster_item ON cluster_item_index(cluster, business_id)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS item_item_similarity (
    item_id TEXT PRIMARY KEY,
    similarity_vector BLOB
)''')

cursor.execute('''CREATE INDEX IF NOT EXISTS idx_item_similarity ON item_item_similarity(item_id)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS business_mapping (
    business_id TEXT PRIMARY KEY,
    business_idx INTEGER
)''')


# Commit the changes
conn.commit()

In [15]:
# insert_user_item_cluster(user_cluster, cluster_business, conn)
insert_cluster_item(cluster_business, conn)
insert_item_vectors(item_similarity_sparse, business_mapping, conn)
insert_mappings(business_mapping, conn)
insert_user_cluster_mapping(df_cluster, conn)

Inserted 50000 / 577061 cluster-item records.
Inserted 250000 / 577061 cluster-item records.
Inserted 500000 / 577061 cluster-item records.
Total 577061 cluster-item records inserted.
Inserted 50000 item vectors...
Total 70727 item vectors inserted.
Inserted 50000 / 70727 business mappings.
Total 70727 business mappings inserted.
Inserted 50000 / 99812 user cluster mappings.
Total 99812 user cluster mappings inserted.


In [16]:
# Close the connection when done
conn.close()

In [17]:
conn = sqlite3.connect(db_path)
optimize_db(conn)

In [18]:
# set the test data label as 1
pos = 3.9
test_data_grouped = test_data.groupby('cluster')['business_id'].apply(list).reset_index()
# test_data_grouped

In [ ]:
# # Retrieve user-business mappings
# def retrieve_business_mapping(conn):
#     cursor = conn.cursor()
#     cursor.execute('''SELECT business_id, business_idx FROM business_mapping''')
#     business_mapping = {row[0]: row[1] for row in cursor.fetchall()}
#     return business_mapping

# fake_business_mapping = retrieve_business_mapping(conn)
# get_top_k_similar_businesses("23qa5SsahSsVhqAVmX6aow", fake_business_mapping, conn, 100)

In [67]:
results = simulate_recommendations(test_data_grouped, business_mapping, conn, k=500, num_users=100)

In [68]:
true_positive, true_negative, false_positive, false_negative, total, total_positive, ranks = check_retrieval_recommendations(results, test_data, test_data_grouped, pos)
evaluation_metric, confusion, background_stats = compute_evaluation_metric(true_positive, true_negative, false_positive, false_negative, total, total_positive, ranks)
print("Evaluation Metric:")
print(evaluation_metric)
print("\nConfusion Matrix:")
print(confusion)
print("\nBackground Statistics:")
print(background_stats)

conn.close()

Evaluation Metric:
   Accuracy  Precision  Recall  F1 Score  F-beta Score  Mean Reciprocal Rank
0    0.6042     0.4893  0.1177    0.1898        0.1388                0.0496

Confusion Matrix:
   True Positive  True Negative  False Positive  False Negative
0            275           3310             287            2061

Background Statistics:
   Total Positive  Total Negative  Total    Ratio
0            2336            3597   5933  0.39373
